МФТИ ФИВТ, Машинное обучение, Никита Волков

# Ранжирование

* Дедлайн **14 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями.
* Кагл: https://inclass.kaggle.com/c/ml-mipt-ranking4
* Инвайт: https://kaggle.com/join/4uthnui9sndi

### Небольшая предыстория

Внезапно очутившись на 4 курсе (как это произошло, вообще не понятно) студенты ФИВТа поняли, что времени стало совсем не хватать. Немного подумав, они решили, что во всем виноваты обезумевшие преподаватели курса Машинное обучение. Было проведено коллективное собрание, на котором было принято решение — попробовать договориться с преподавателями. Однако, все попытки примирения были отвергнуты. И тогда на очередном собрании (вместо выполнения очередного ДЗ ;)) было принято решение сделать вопросно-ответную систему для студентов. Например, она сможет четко отвечать на вопросы следующего типа ”Когда дедлайн по очередному огромному заданию?”, ”Что нужно сделать в задании?”, а вопрос ”Сколько суток нужно потратить, чтобы выполнить очередное ДЗ” даже никто и задавать не будет — никто их не делает, кроме самих преподавателей. А на вопрос ”Сможем ли мы переубедить Арсения Ашуху?” система ответит картинкой:

<img width=400 src="./pic.png">

Для каждого вопроса студенты смогли написать систему, которая подбирает наиболее подходящие ответы в интернете. А вот, как из упорядочить, они не знают, поскольку прогуляли лекцию по ранжированию. К счастью, было записано видео этой лекции, так что вы сможете им помочь.

А теперь серьезно...

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

### Бейзлайн

In [12]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import math

Считываем все запросы из test

In [29]:
with open('data/queries_test.txt') as f:
# with open('data/queries_train.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

queries[:5]

['Почему нельзя неподошедшего тебе мужика просто аккуратненько вернуть обратно маме?',
 'Ты никогда не замечал, какие все вокруг оптимисты. когда проблема касается тебя?',
 'В какой момент нужно сказать привет когда навстречу идет знакомый?',
 'Кто знает, какие успокоительные лекарства принимает медведь в мультике про Машу? мне тоже такие надо.',
 'Таджики на уроке физкультуры по команде строимся,по строили новый дом .']

По очереди загружаем все документы, считая для каждого TF и сортируя для каждого запроса. Это и есть скор в бейзлайне.

In [30]:
ft = open('data/test.txt')
# ft = open('data/train.txt')
ft.readline()
pairs = ft.readlines()
ft.close()

pairs[:5]

['101,0,https://vk.com/id377383444\n',
 '101,1,http://xn----btbgrjageubg1a.xn--p1ai/420-skuchat\n',
 '101,2,http://jiznenno.ru/vykhod-iz-treugolnika-karpmana/\n',
 '101,3,http://iknigi.net/avtor-aleksandra-marinina/113760-obratnaya-sila-tom-2-19651982-aleksandra-marinina/read/page-1.html\n',
 '101,4,http://fb.ru/article/270014/esli-tufli-veliki-chto-delat-sposobyi-resheniya-problemyi\n']

In [4]:
def get_query_words(query):
    words = query.lower().split(' ')
    words_filtered = ["".join(list(filter(lambda symbol: symbol.isalpha(), word))) for word in words]
    words_russian = ["".join(list(filter(lambda symbol: symbol >= 'а' and symbol <= 'я', word))) for word in words_filtered]
    words_not_empty = list(filter(lambda word: word != "", words_russian))
    return words_not_empty

In [6]:
for query in queries[:5]:
    print(get_query_words(query))

['почему', 'нельзя', 'неподошедшего', 'тебе', 'мужика', 'просто', 'аккуратненько', 'вернуть', 'обратно', 'маме']
['ты', 'никогда', 'не', 'замечал', 'какие', 'все', 'вокруг', 'оптимисты', 'когда', 'проблема', 'касается', 'тебя']
['в', 'какой', 'момент', 'нужно', 'сказать', 'привет', 'когда', 'навстречу', 'идет', 'знакомый']
['кто', 'знает', 'какие', 'успокоительные', 'лекарства', 'принимает', 'медведь', 'в', 'мультике', 'про', 'машу', 'мне', 'тоже', 'такие', 'надо']
['таджики', 'на', 'уроке', 'физкультуры', 'по', 'команде', 'строимсяпо', 'строили', 'новый', 'дом']


In [31]:
def count_df(texts, query):
    texts_set = [set(text) for text in texts]
    df = []
    for word in query:
        k = 0
        for text_set in texts_set:
            if word in text_set:
                k += 1
        cur_df = (k + 0.0) / len(texts)
        df.append(cur_df)
#         print(word, cur_df, k, len(texts))
    return df

In [32]:
def find_relevs(texts, query):
    relevs = []
    df = count_df(texts, query)
    for text in texts:
        frec = 0
        if len(text) > 0:
            for index in range(len(query)):
                word = query[index]
                idf = math.log(1.0 / df[index]) if df[index] > 0 else 0
                frec += (text == word).mean() * idf
#         print(frec)
        relevs.append(frec)
    return relevs

In [34]:
MAX_ITERATIONS_NUMBER = 0
# MAX_ITERATIONS_NUMBER = 200

def process_file(pairs, queries, output):
    qids, docids, texts = [], [], []
    iteration_number = 0
    for line in pairs:
        if MAX_ITERATIONS_NUMBER != 0 and iteration_number > MAX_ITERATIONS_NUMBER:
            break
        iteration_number += 1
            
        print(line)
        qid, docid, url = line.split(',')[:3]
#         qid, docid, url = line.split(',')[1:4]
        qid, docid = int(qid), int(docid)
        
        if len(qids) > 0 and qid != qids[-1]:
            relevs = find_relevs(texts, get_query_words(queries[qids[-1] - 101]))
            order = np.argsort(relevs)[::-1]
            qids = np.array(qids)[order]
            docids = np.array(docids)[order]

            for i in range(len(qids)):
                output.write('{},{}\n'.format(qids[i], docids[i]))

            qids, docids, texts = [], [], []
            
        try:
            text = ' '.join(BeautifulSoup(urlopen(url, timeout=10), 'lxml').stripped_strings)
            text = np.array(get_query_words(text))
#             print(text)
        except Exception as e:
            pass
                
        qids.append(qid)
        docids.append(docid)
        texts.append(text)
        
    if len(qids) > 0:
        relevs = find_relevs(texts, get_query_words(queries[qids[-1] - 101]))
        order = np.argsort(relevs)[::-1]
        qids = np.array(qids)[order]
        docids = np.array(docids)[order]

        for i in range(len(qids)):
            output.write('{},{}\n'.format(qids[i], docids[i]))

        qids, docids, texts = [], [], []

In [35]:
%%time

fb = open('results/result_2_1_tfidf.txt', 'w')
# fb = open('results/result_2_1_train_tfidf.txt', 'w')
fb.write('QueryId,DocumentId\n')

process_file(pairs, queries, fb)

fb.close()

101,0,https://vk.com/id377383444

101,1,http://xn----btbgrjageubg1a.xn--p1ai/420-skuchat

101,2,http://jiznenno.ru/vykhod-iz-treugolnika-karpmana/

101,3,http://iknigi.net/avtor-aleksandra-marinina/113760-obratnaya-sila-tom-2-19651982-aleksandra-marinina/read/page-1.html

101,4,http://fb.ru/article/270014/esli-tufli-veliki-chto-delat-sposobyi-resheniya-problemyi

101,5,https://new.vk.com/wall-56106344_4734187

101,6,https://prodoctorov.ru/krasnodar/lpu/8343-uro-pro/

101,7,http://garri-potter-life.narod.ru/online/hp_kamen.pdf

101,8,http://otvet.mail.ru/mail/lady.dobrota777

101,9,http://24advokat.ru/cat/v_kakih_sluchayah_mojno_vernut_telefon/

101,10,http://www.voprosnik.ru/forum/poll/948567/%D0%97%D0%B0%D0%BA%D0%BE%D0%BD%D0%BE%D0%BC%D0%B5%D1%80%D0%BD%D0%BE%D1%81%D1%82%D1%8C-%D0%B8%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D1%85-%D1%8F%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B9-%D0%BE%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%BE-%D0%BF%D1%80%D0%BE%D0%BF%D0%BE%D1%80%D1%86%D0%B8%D0%BE

Если у вас не загружаются некоторые страницы, подумайте, как с этим можно бороться. Например, посмотрите код с семинара и из предыдущего ДЗ. Если и это не помогает, то нужно подумать, что делать в этой ситуации.

### Кажется, это последнее задание

Свобода!

<img width=200 src="./last.jpg">

А, нет, еще экзамен...